In [1]:
import pandas
import datetime
import numpy
import multiprocessing

import rt
import rt_sct_utils
# import geo_locate
import plot_utils

import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.dates import date2num, DateFormatter

import time

In [2]:
sel_beam = 11

In [3]:
cpu_count = multiprocessing.cpu_count()
# radar params
sTime = datetime.datetime(2018,2,5)#datetime.datetime(2010,10,13)
eTime = datetime.datetime(2018,2,6)#datetime.datetime(2010,10,14)
date_plot = datetime.datetime(2018,2,5, 20)#datetime.datetime(2010,10,13,20)
radar = 'cvw'

In [4]:
start_time = time.time()
rto = rt.RtRun(sTime=sTime, eTime=eTime,\
               rCode=radar, beam=sel_beam,\
               outDir='/tmp/', nprocs=cpu_count,\
              nhops=2,freq=12)
print("--- %s seconds ---" % (time.time() - start_time))

In:: ['mpiexec', '-n', '8', '/home/bharat/Documents/code/raydarn/rtFort', '/tmp/rtrun.2018036_12.inp', '/tmp/', '2018036_12']
Exit code:: 0
Returned:: 
 Input file: /tmp/rtrun.2018036_12.inp                                                                                                                                                                                                                                 
 Output directory: /tmp/                                                                                                                                                                                                                                                     
 File extension: 2018036_12
   37.0999985      -77.9499969      -41.6199989      -41.6199989       1.00000000       5.00000000       60.0000000      0.100000001       12.0000000               2        2018         205   25.0000000       49.0000000       1.00000000       0.00000000       0.00000000     2018036_12N

In [5]:
rto.readRays()
rdict = rto.rays.paths
# rdict[date_plot][sel_beam][6.5].keys()

/tmp/rays.2018036_12.dat header: 
nhour      :: 25
nazim      :: 1
nelev      :: 551
tlat       :: 37.099998474121094
tlon       :: -77.94999694824219
saz        :: -41.619998931884766
eaz        :: -41.619998931884766
daz        :: 1.0
sel        :: 5.0
eel        :: 60.0
del        :: 0.10000000149011612
freq       :: 12.0
nhop       :: 2
year       :: 2018
mmdd       :: 205
shour      :: 25.0
ehour      :: 49.0
dhour      :: 1.0
hmf2       :: 0.0
nmf2       :: 0.0
fext       :: b'2018036_12'
outdir     :: b'None'
indir      :: b'/tmp/'


In [6]:
rto.readEdens()
edenstht = rto.ionos.edens[date_plot][sel_beam]['th']#dict_keys(['th', 'nel', 'dip'])
edensArr = rto.ionos.edens[date_plot][sel_beam]['nel']

/tmp/edens.2018036_12.dat header: 
nhour      :: 25
nazim      :: 1
nelev      :: 551
tlat       :: 37.099998474121094
tlon       :: -77.94999694824219
saz        :: -41.619998931884766
eaz        :: -41.619998931884766
daz        :: 1.0
sel        :: 5.0
eel        :: 60.0
del        :: 0.10000000149011612
freq       :: 12.0
nhop       :: 2
year       :: 2018
mmdd       :: 205
shour      :: 25.0
ehour      :: 49.0
dhour      :: 1.0
hmf2       :: 0.0
nmf2       :: 0.0
fext       :: b'2018036_12'
outdir     :: b'None'
indir      :: b'/tmp/'


In [ ]:
rto.readScatter()

/tmp/gscat.2018036_12.dat header: 
nhour      :: 25
nazim      :: 1
nelev      :: 551
tlat       :: 37.099998474121094
tlon       :: -77.94999694824219
saz        :: -41.619998931884766
eaz        :: -41.619998931884766
daz        :: 1.0
sel        :: 5.0
eel        :: 60.0
del        :: 0.10000000149011612
freq       :: 12.0
nhop       :: 2
year       :: 2018
mmdd       :: 205
shour      :: 25.0
ehour      :: 49.0
dhour      :: 1.0
hmf2       :: 0.0
nmf2       :: 0.0
fext       :: b'2018036_12'
outdir     :: b'None'
indir      :: b'/tmp/'


In [ ]:
fig = plt.figure(figsize=(16, 6))
ax, aax = plot_utils.curved_earth_axes()
X, Y = numpy.meshgrid(edenstht, ax.Re + numpy.linspace(60,560,250))
im = aax.pcolormesh(X, Y,  edensArr, cmap="cividis")
cbax = plot_utils.add_cbar(im, ax)
_ = cbax.set_ylabel(r"N$_{el}$ [$m^{-3}$]", fontsize=14)
ax.set_title(date_plot.strftime("%Y-%m-%d %H:%M") + " ("+radar+")", fontsize=14)
ax.set_ylabel(r"Alt. [km]", size=16)
ax.set_xlabel(r"Ground range [km]", size=16)

# plot rays
# plot rays
for _el in rdict[date_plot][sel_beam].keys():
    rays = rdict[date_plot][sel_beam][_el]
    aax.plot(rays['th'], numpy.array(rays['r'])*1e-3, c='#9658B1', 
                            zorder=8, linewidth=1.)
# plot rays
# plot rays

# Plot ranges
# Plot ranges
range_markers = [0] + list(numpy.arange(180, 5000, 225))
x, y = [], []


for _el in rdict[date_plot][sel_beam].keys():
    rays = rdict[date_plot][sel_beam][_el]
    grans = numpy.array(rays['gran'])*1e-3
    th = numpy.array(rays['th'])
    r = numpy.array(rays['r'])
    for rm in range_markers:
        inds = (grans >= rm)
        if inds.any():
            x.append( th[inds][0] )
            y.append( r[inds][0]*1e-3 )
    aax.scatter(x, y, color="darkgray",s=0.25, zorder=9, alpha=0.4)
# Plot ranges
# Plot ranges

# Plot grnd.scat
# Plot grnd.scat

for _el in rto.scatter.gsc[date_plot][sel_beam].keys():
    gscat = rto.scatter.gsc[date_plot][sel_beam][_el]
    if gscat is not None:
#         print(gscat.keys())
        aax.scatter(gscat['th'], ax.Re*numpy.ones(gscat['th'].shape), 
                        color='k', zorder=10)
# Plot grnd.scat
# Plot grnd.scat

# Plot ion.scat
# Plot ion.scat

for _el in rto.scatter.isc[date_plot][sel_beam].keys():
    ionos = rto.scatter.isc[date_plot][sel_beam][_el]

    if ionos['nstp'] <= 0:
        continue
    t = ionos['th']
    r = ionos['r']*1e-3
    spts = numpy.array([t, r]).T.reshape(-1, 1, 2)
    h = ionos['h']*1e-3
    rel = numpy.radians( ionos['rel'] )
    r = numpy.sqrt( r**2 + h**2 + 2*r*h*numpy.sin( rel ) )
    t = t + numpy.arcsin( h/r * numpy.cos( rel ) )
    epts = numpy.array([t, r]).T.reshape(-1, 1, 2)
    segments = numpy.concatenate([spts, epts], axis=1)
    lcol = LineCollection( segments, zorder=10,linewidths=5. )
    
    _ = lcol.set_color('k')
    aax.add_collection( lcol )

# Plot ion.scat
# Plot ion.scat
# fig.savefig("iri_test.png", dpi=200, bbox_inches="tight")

In [ ]:
sct_obj = rt_sct_utils.RT_SCT(rto)
gnd_df = sct_obj.get_gnd_sct_df()
# dict_keys(['r', 'th', 'gran', 'lat', 'lon', 'hops'])
iono_df = sct_obj.get_iono_sct_df()
# dict_keys(['nstp', 'r', 'th', 'gran', 'rel', 'w', 'nr', 'lat', 'lon', 'h', 'hops'])
#http://davit1.ece.vt.edu/doc/rt/rt_read_ranges-code.html

In [ ]:
# gnd_df.groupby('hop').size()#[gnd_df['hop'] == 0.5]
print(iono_df.head())
print("-------------")
print(gnd_df.head())

In [ ]:
# prepare iono_df for plotting
# prepare iono_df for plotting
plot_param = "lag_power"
vmin=-30.
vmax=0.

iono_plot_df = iono_df[ ["date", "range",\
                        plot_param] ].pivot( "date", "range" )

iono_time_vals = iono_plot_df.index.values
iono_range_vals = iono_plot_df.columns.levels[1].values
iono_time_cntr, iono_rng_cntr  = numpy.meshgrid( iono_time_vals, iono_range_vals )
# Mask the nan values! pcolormesh can't handle them well!
iono_pwr_vals = numpy.ma.masked_where(\
                numpy.isnan(iono_plot_df[plot_param].values),\
                iono_plot_df[plot_param].values)
# prepare iono_df for plotting
# prepare iono_df for plotting

# prepare gnd_df for plotting
# prepare gnd_df for plotting
# sel_gnd_df = gnd_df[(gnd_df["median_alt"] > 1) & gnd_df["hop"] == 1]
# sel_gnd_df["median_alt"] = sel_gnd_df["median_alt"] - 6370.
gnd_plot_df = gnd_df[ ["date", "range",\
                        plot_param] ].pivot( "date", "range" )

gnd_time_vals = gnd_plot_df.index.values
gnd_range_vals = gnd_plot_df.columns.levels[1].values
gnd_time_cntr, gnd_rng_cntr  = numpy.meshgrid( gnd_time_vals, gnd_range_vals )
# Mask the nan values! pcolormesh can't handle them well!
gnd_pwr_vals = numpy.ma.masked_where(\
                numpy.isnan(gnd_plot_df[plot_param].values),\
                gnd_plot_df[plot_param].values)
# prepare gnd_df for plotting
# prepare gnd_df for plotting

In [ ]:
plt.style.use("fivethirtyeight")

fig = plt.figure(figsize=(11,8))
ax = fig.add_subplot(111)

iono_rti_plot = ax.pcolormesh(iono_time_cntr.T , iono_rng_cntr.T, iono_pwr_vals,\
                        cmap="viridis", vmin=vmin,vmax=vmax)

gnd_rti_plot = ax.pcolormesh(gnd_time_cntr.T , gnd_rng_cntr.T, gnd_pwr_vals,\
                        cmap="viridis", vmin=vmin,vmax=vmax)

ax.get_xaxis().set_major_formatter(DateFormatter('%H'))

ax.set_ylabel("Range [Km]")
ax.set_xlabel("UT HOUR")
ax.set_title(date_plot.strftime("%Y-%m-%d"))

plt.xticks(rotation=45)
cb = fig.colorbar(gnd_rti_plot)
cb.set_label("Relative power [dB]")

ax.grid()


In [ ]:
gnd_df[gnd_df["date"] >= sTime+datetime.timedelta(hours=13)]["date"].unique()

In [ ]:
sel_gnd_df = gnd_df[(gnd_df["median_alt"] > 1) & gnd_df["hop"] == 1]
sel_gnd_df["median_alt"] = sel_gnd_df["median_alt"] - 6370.
grp_gnd_df = sel_gnd_df.groupby(["date", "azim"]).min().reset_index()

In [ ]:
plt.style.use("fivethirtyeight")

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(grp_gnd_df["date"], grp_gnd_df["median_alt"])
ax.scatter(grp_gnd_df["date"], grp_gnd_df["median_alt"], s=100, c='r')

ax.set_xlim([datetime.datetime(2016,1,1,13), datetime.datetime(2016,1,2)])
ax.set_ylim([100,240])

ax.get_xaxis().set_major_formatter(DateFormatter('%H:%M'))

In [ ]:
grp_gnd_df

In [ ]:
(227-185)/2